In [1]:
# Companies House Data Retrieval Using Basic Stream Request

# This notebook demonstrates how to use basic authentication with an API key for the Companies House API to retrieve data on dairy and cattle farms, and prepare the data for social network analysis.
# 
# The Companies House API provides access to a wealth of data on companies registered in the UK, including information on company officers, financials, and more. In this example, we focus on retrieving data on dairy and cattle farms using SIC codes.
#
# The steps include:
# - Installing the required libraries
# - Importing the necessary libraries
# - Defining a function to make API requests
# - Fetching companies by SIC code
# - Fetching detailed company information
# - Retrieving and analyzing officer data
# - Preparing data for social network analysis
# - Saving the data to a CSV file

In [2]:
### 1. Install Required Libraries

# !pip install requests jupyter notebook pandas


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/dd/49/de869130028fb8d90e25da3b7d8fb13e40f5afa4c4af1781583eb1ff3839/pandas-2.2.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata
  Using cached pandas-2.2.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata (19 kB)
  Obtaining dependency information for numpy>=1.26.0 from https://files.pythonhosted.org/packages/a0/97/61ed64cedc1b94a7939e3ab3db587822320d90a77bef70fcb586ea7c1931/numpy-2.0.0-cp312-cp312-macosx_14_0_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 215.8 kB/s eta 0:00:00 0:00:01m
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b

In [3]:
### 2. Import Required Libraries

import requests
import pandas as pd
import time

# Replace 'your_api_key' with your actual API key
API_KEY = '669fcc9f-7d0f-4566-a9c5-002f4a3f2aee'  
BASE_API_URL = 'https://api-sandbox.company-information.service.gov.uk'

In [4]:
### 3. Define the Function to Perform API Requests

def make_request(endpoint, params=None):
    headers = {
        'Authorization': f'Basic {API_KEY}'
    }
    url = f"{BASE_API_URL}{endpoint}"
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()


In [5]:
### 4. Fetch Companies by SIC Code

#  remember  we  only have a maximum of 600 requests per 5 minutes
#  so we have to be careful with the number of requests we make

def search_companies_by_sic(sic_codes, max_results=100): # max_results per SIC code
    companies = []
    for sic_code in sic_codes:
        start_index = 0
        while start_index < max_results:
            params = {
                'q': f'SIC:{sic_code}',
                'start_index': start_index,
                'items_per_page': 100 # Maximum items per page
            }
            data = make_request('/search/companies', params)
            companies.extend(data.get('items', []))
            if 'total_results' in data:
                total_results = data['total_results']
            else:
                break
            start_index += 100
            if start_index >= total_results:
                break
            time.sleep(1)  # To respect rate limits
    return companies

# SIC Codes for dairy and cattle farms
sic_codes = ['01410', '01420']
companies = search_companies_by_sic(sic_codes)
print(f"Found {len(companies)} companies.")


HTTPError: 400 Client Error: Bad Request for url: https://api-sandbox.company-information.service.gov.uk/search/companies?q=SIC%3A01410&start_index=0&items_per_page=100

In [ ]:
### 5. Fetch Detailed Company Information

def fetch_company_details(company_number):
    endpoint = f'/company/{company_number}'
    company_details = make_request(endpoint)
    return company_details


In [ ]:
### 6. Retrieve and Analyze Officer Data

def fetch_company_officers(company_number):
    endpoint = f'/company/{company_number}/officers'
    officers = make_request(endpoint)
    return officers.get('items', [])

# Example usage:
company_number = companies[0]['company_number']
officers = fetch_company_officers(company_number)
print(officers)


In [ ]:
### 7. Prepare Data for Social Network Analysis

def prepare_sna_data(companies):
    sna_data = []
    for company in companies:
        company_number = company['company_number']
        company_details = fetch_company_details(company_number)
        officers = fetch_company_officers(company_number)

        for officer in officers:
            sna_data.append({
                'company_name': company_details.get('company_name'),
                'company_number': company_number,
                'officer_name': officer.get('name'),
                'officer_role': officer.get('officer_role'),
                'appointed_on': officer.get('appointed_on'),
                'resigned_on': officer.get('resigned_on')
            })
        
        # Rate limiting
        time.sleep(1)  # To respect rate limits
        if len(sna_data) >= 600:
            time.sleep(300)  # To avoid exceeding rate limit
    
    return pd.DataFrame(sna_data)

sna_df = prepare_sna_data(companies)
sna_df.head()


In [ ]:
### 8. Save the Data to CSV

sna_df.to_csv('sna_data.csv', index=False)
print("Data saved to 'sna_data.csv'")
    

In [ ]:
# This markdown content includes the Python code you can use to authenticate with the Companies House API using basic stream requests, fetch data on companies, and prepare it for social network analysis.
# 
# Make sure to replace 'your_api_key' with your actual API key before running the code.
#
# The code demonstrates how to:
# - Install the required libraries
# - Import the necessary libraries
# - Define a function to make API requests
# - Fetch companies by SIC code
# - Fetch detailed company information
# - Retrieve and analyze officer data
# - Prepare data for social network analysis
# - Save the data to a CSV file
